In [1]:
import datetime as dt
import pytz
from matplotlib import pyplot as plt
from django.db.models.query_utils import Q
import pandas as pd
from tqdm.notebook import tqdm
import openpyxl

In [2]:
!cat .env

STAGE=PRODUCTION


In [13]:
# road_companies = ['Future',
#  'Arteris Intervias',
#  'Neovia Tijucas',
#  'VF Gomes - Vila Bela',
#  'Entrevias',
#  'Neovia Rancho Queimado 080/2021',
#  'Afirma Engenharia Viária',
#  'Neovia Bocaína do Sul 329/2018',
#  'VF Gomes - Confresa',
#  'ELO Ecopistas',
#  'Arteris Litoral Sul',
#  'Neovia Contorno Sul',
#  'Neovia Diamantino',
#  'Neovia Vacaria',
#  'VF Gomes - Porto Espiridião',
#  'CART',
#  'Dalba Guarapuava',
# 'Neovia Oeste',
# 'ELO Rota das Bandeiras',
# 'Via Brasil',
# 'Arteris Fernão Dias - Conserva',
#   'Arteris Fluminense - Conserva',
#    'Arteris Litoral Sul - Conserva' ,
#     'Arteris Régis Bittencourt - Conserva' ,
#         'Dalba Foz do Iguaçu', 'Eixo SP SISOAE', 'Arteris Via Paulista - SISOBRAS', 'Neovia Ponte Serrada',
#         'Neovia Régis Bittencourt', 'Rota do Oeste', 'VF Gomes - BR158', 'ELO Via Lagos', 'Garcia Monteiro', 
#                   'Arteris Via Paulista - SISOBRAS', 'Televale','Arteris Fernão Dias', 'Arteris Fluminense', 'Arteris Planalto Sul - Conserva','Arteris Régis Bittencourt',
#                   'Neovia General Carneiro','Eixo SP', 'Neovia Cto Biguaçu', 'VF Gomes - Nova Lacerda','Neovia Planalto Sul',
#                   'Neovia Alcinópolis', 'ELO Rodovias Tietê', 'VF Gomes - BR010', 'VF Gomes - BR010', 'Sanson', 'ELO Rodovias dos Tamoios',
#                   'Neovia Lages 652/2019', 'Pavicolet', 'Neovia Lagoa Vermelha','Neovia Araquari 886/2020','Arteris Planalto Sul', 'Entrevias SISOAE', 'Eixo SP - Fx. Dom. | Qualidade | Topografia',
#                  'ELO Ecovias dos Imigrantes', 'Televale', 'Arteris Planalto Sul','Arteris Régis Bittencourt']
road_companies = ['Entrevias']

In [14]:
today = dt.date.today().strftime('%m')
today

'09'

In [15]:
ano = dt.date.today().year
ano

2023

In [16]:
months_number = [i for i in range(1, int(today))]

months_number

[1, 2, 3, 4, 5, 6, 7, 8]

In [17]:
def quantitativos_v2(months_name,company):
    sem_qtt =[]

    for n in months_name:
        com_qtt = 0
        apontamentos_totais = 0
        for occ in OccurrenceType.objects.filter(company__name = company).exclude(occurrence_kind='2'):
            autofill_keys = []
            autofill_keys = [field["apiName"] for field in occ.form_fields["fields"] if "autofill" in field.keys()]
            size_occ=len(autofill_keys)            
            reportings=Reporting.objects.filter(occurrence_type=occ, created_at__month = n, company__name = company, created_at__year=ano).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='cartcs').only('form_data', 'occurrence_type').select_related('occurrence_type')
            reportings_keys=[len(a.keys()) for a in list(reportings.values_list('form_data', flat=True))]            
            com_qtt += len([a for a in reportings_keys if a > size_occ])
            apontamentos_totais += reportings.count()
        sem_qtt.append((apontamentos_totais - com_qtt)/apontamentos_totais if apontamentos_totais != 0 else 0)
    return sem_qtt

In [18]:
def dias_de_uso_v2(months_name, company):
    
    qtd_dias = []
    dias = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, mes in enumerate(months_name):

        a= []
        for n in range(1, dias[index]):
            a.append(Reporting.objects.filter(company__name = company, created_at__year=ano, created_at__month = mes,created_at__day=n).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='neoviacs').exclude(created_by__username='arteriscs').only('uuid').count())
        qtd_dias.append(a)

    
    quantidade = []
    porcen_dias= []
    
    for qtd in qtd_dias:
        quantidade.append(len([x for x in qtd if x != 0]))
        
    
    for index, qtd in enumerate(quantidade):
        b = qtd/dias[index]
        porcen_dias.append(b)
    return porcen_dias

In [19]:
Reporting.objects.filter(company__name = "Entrevias", created_at__year=2023, created_at__month = 7,created_at__day=31).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='neoviacs').exclude(created_by__username='arteriscs').only('uuid').count()

3143

In [20]:
def equipes_ativas_v2(months_name, company):
    team_name = list(Firm.objects.filter(company__name = company).values_list('name', flat=True))
    
#     team_name = []
#     for i in range(len(teams)):
#         team_name.append(teams[i].name)
    
    equipes=[]
    for team in team_name:
        for i in months_name:
            equipes.append(Reporting.objects.filter(company__name = company, created_at__year=2023, created_at__month= i,firm__name = team).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='neoviacs').exclude(created_by__username='arteriscs').only('uuid').count())
    
#     equipe=[]
#     a =0
#     b = 11
#     for i in range(0, len(team_name)):
#         equipe.append(equipes[a:b])
#         a += 11
#         b+=11
    
    soma = []
    for team in team_name:
        soma.append(Reporting.objects.filter(company__name = company, created_at__year=2023,firm__name = team).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='neoviacs').exclude(created_by__username='arteriscs').only('uuid').count())

#     top = []
#     if len(soma) < 5:
#         top = sorted(soma)
#     else:    
#         ordenada = sorted(soma, reverse = True)
#         top = ordenada[:5]
#     top=sorted(soma[:5])
    
    porcen = []
    for month in months_name:
        firm_moves=[]
        objects=['Reporting', 'Job']
        for firm in team_name:
            num_moves = 0
            for item in objects:
                num_moves+=eval(item).history.filter(history_date__year=2023, history_date__month=month, company__name= company, firm__name=firm).only('uuid').count()
            firm_moves.append((num_moves, firm))

        firm_moves.sort(key=lambda tup: tup[0], reverse=True)
        active_firms=len([firm for firm in firm_moves if firm[0]>0])
        porcen.append(active_firms/len(firm_moves) if len(firm_moves) != 0 else 0)
    
    return porcen

In [21]:
def variacao(months):
    taxa_variacao = []
    for i,m in enumerate(months):
        y = months[i]
        b = months
        c = b.index(y) 
        if c == 0:
            taxa_variacao.append(0)
        if y > b[c-1] and c != 0:
            m = (y-b[c-1])/y
            taxa_variacao.append(m)
        if y == b[c-1] and c != 0:
            i = y*0
            taxa_variacao.append(i)
        if y < b[c-1] and c!= 0 :
            n = (y -b[c-1])/b[c-1]
            taxa_variacao.append(n)
    return taxa_variacao


In [22]:
def qtt_nota(nota):
    a = 0
    if nota <0.05:
        a = 1
    if nota >= 0.05 and nota <0.15:
        a = 0.65
    if nota >= 0.15 and nota < 0.3:
        a = 0.3
    if nota >= 0.3:
        a = 0
    return a

In [23]:
def uso_nota(nota):
    a = 0
    if nota <0.4:
        a = 0
    if nota >= 0.4 and nota <0.65:
        a = 0.3
    if nota >= 0.65 and nota < 0.8:
        a = 0.65
    if nota > 0.8:
        a = 1
    return a

In [24]:
def equipe_nota(nota):
    a = 0
    if nota > 0.6:
        a = 1
    if nota >0.45 and nota <=0.6:
        a = 0.65
    if nota > 0.3 and nota <= 0.45:
        a = 0.3
    if nota <= 0.3:
        a = 0
    return a

In [25]:
def variacao_nota(nota):
    a= 0
    if nota >= 0.25:
        a = 1
    if nota >= 0.15 and nota <0.25:
        a = 0.75
    if nota >= -0.15 and nota < 0.15:
        a = 0.5
    if nota >= -0.25 and nota < -0.15:
        a = 0.5
    if nota < -0.25:
        a = 0
    return a

In [26]:
hs_dict = {}
uso_sistema = []
sem_quantitativos = []
equipes = []
variacao_dias = []
parametros = ['Dias de uso', 'Sem quantitativos', 'Equipes Ativas', 'Variacao dias']
for company in tqdm(road_companies):
    months=[]
    print(company)
    for n in months_number:
        months.append(Reporting.objects.filter(company__name = company, created_at__year=2023, created_at__month=n).exclude(occurrence_type__occurrence_kind='2').exclude(created_by__username='rlcs').exclude(created_by__username='neoviacs').exclude(created_by__username='arteriscs').only('uuid').count())
    print(months)
    uso_sistema.append(dias_de_uso_v2(months_number, company))
    sem_quantitativos.append(quantitativos_v2(months_number, company))
    equipes.append(equipes_ativas_v2(months_number, company))
    variacao_dias.append(variacao(months)) 
    if not hs_dict.get(company):
        hs_dict[company] = {}
        hs_dict[company]['Dias de uso'] = dias_de_uso_v2(months_number, company)[-1]
        hs_dict[company]['Sem quantitativos'] = quantitativos_v2(months_number, company)[-1]
        hs_dict[company]['Equipes ativas'] = equipes_ativas_v2(months_number, company)[-1]
        hs_dict[company]['Variação dias'] =  variacao(months)[-1]
    print(f' dias de uso: {dias_de_uso_v2(months_number, company)} \n sem quantitativos: {quantitativos_v2(months_number, company)} \n equipes ativas: {equipes_ativas_v2(months_number, company)} \n variação dias: {variacao(months)} ')

  0%|          | 0/1 [00:00<?, ?it/s]

Entrevias
[7153, 8711, 13719, 6011, 15204, 13530, 8839, 12460]
 dias de uso: [0.967741935483871, 0.9642857142857143, 0.967741935483871, 0.9666666666666667, 0.9354838709677419, 0.9666666666666667, 0.967741935483871, 0.967741935483871] 
 sem quantitativos: [0.4087795330630505, 0.3164963838824475, 0.0989868066185582, 0.17933788055232075, 0.1637726913970008, 0.19903917220990391, 0.15273220952596447, 0.09606741573033707] 
 equipes ativas: [0.41304347826086957, 0.43478260869565216, 0.5869565217391305, 0.6413043478260869, 0.6086956521739131, 0.6304347826086957, 0.6195652173913043, 0.6413043478260869] 
 variação dias: [0, 0.17885432212145563, 0.36504118375974925, -0.561848531234055, 0.6046435148645093, -0.1101026045777427, -0.34671101256467113, 0.2906099518459069] 


In [27]:
hs_dict

{'Entrevias': {'Dias de uso': 0.967741935483871,
  'Sem quantitativos': 0.09606741573033707,
  'Equipes ativas': 0.6413043478260869,
  'Variação dias': 0.2906099518459069}}

In [19]:
#tabela = pd.DataFrame(hs_dict)

In [20]:
#tabela

In [21]:
#tabela.to_excel('analise_hs_ELO.xlsx')

In [28]:
hs_month = []
hs_month_export = {}
for i in tqdm(range(len(months_number))):
    hs = []
    for n in range(len(road_companies)):
        print(road_companies[n])
        if uso_sistema[n][i] == 0:
            hs.append(0)
        else:
            hs.append(35*(uso_nota(uso_sistema[n][i])) + 15*(qtt_nota(sem_quantitativos[n][i])) + 15*(equipe_nota(equipes[n][i]))+ 35*(variacao_nota(variacao_dias[n][i])))
        if not hs_month_export.get(road_companies[n]):
            hs_month_export[road_companies[n]] = []
        hs_month_export[road_companies[n]].append(hs[-1])


    hs_month.append(hs)

  0%|          | 0/8 [00:00<?, ?it/s]

Entrevias
Entrevias
Entrevias
Entrevias
Entrevias
Entrevias
Entrevias
Entrevias


In [29]:
hs_month_export

{'Entrevias': [57.0, 65.75, 89.5, 54.5, 89.5, 72.0, 54.5, 94.75]}

In [24]:
#for i in hs_month_export.items():
 #   print(i[0],i[1][-1])

In [32]:
hs_novo = []
for i in hs_month_export.values():
    hs_novo.append(i[-1])

In [33]:
hs_novo

[66.75]

In [26]:
hs_atencao = {}
for i in hs_month_export:
    if hs_month_export[i][-1] < 30:
        print(i,hs_dict[i])
        if not hs_atencao.get(i):
            hs_atencao[i] =hs_dict[i] 
        
    